# VLASS and GALEX source matches to X-LoVoCCS BCG candidates

## Import statements

In [1]:
from astroquery.vizier import Vizier
from astropy.units import Quantity
from astropy.coordinates import SkyCoord
from IPython.display import display

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import os

## Setting up directories

In [2]:
out_dir = 'outputs/vlass_galex_crossmatches/'
os.makedirs(out_dir, exist_ok=True)

## Defining useful constants

In [3]:
VLASS_SEARCH_RAD = Quantity(10, 'arcsec')

GALEX_SEARCH_RAD = Quantity(10, 'arcsec')

## Defining useful functions

In [4]:
def vlass_search(coords_to_search):
    bcg_cand_vlass = []

    # This will be replaced with the columns pulled from the catalog (plus some extras) later on
    cols = None
    
    # We iterate because even though .query_region is perfectly happy taking a set of coordinates, the returned table 
    #  does not associate particular catalog entries with a particular coordinate that we passed. We do want that information, and
    #  while we could reconstruct it after a query with all coordinates passed, we are going to do it by querying on a galaxy-by-galaxy
    #  basis
    for cur_coord_ind, cur_coord in enumerate(coords_to_search):
        rel_name = bcg_cand_samp['cluster_name'].values[cur_coord_ind]
        
        # Possible there will be NaN coordinates, need to account for that
        if np.isnan(cur_coord.ra.value):
            continue
    
        # Performs the actual search through the catalog
        cur_vlass_res = vlass_cat.query_region(cur_coord, radius=VLASS_SEARCH_RAD)
        
        # Also useful to know if the candidate is within the nominal coverage of the catalog - doesn't guarantee that a 
        #  section of the data hasn't been cut out for some reason, but big picture gives us an idea if there might be coverage
        if cur_coord.dec >= Quantity(-40, 'deg'):
            nominal_coverage = True
        else:
            nominal_coverage = False
    
        # Checking and storing results
        if len(cur_vlass_res) > 0:
            # The key we pass at the end is to retrieve a specific table of matches (some catalogs have multiple tables)
            cur_vlass_res = cur_vlass_res['J/ApJS/255/30/comp'].to_pandas()
            # Make sure the cluster name is in the dataframe
            cur_vlass_res.insert(0, 'cluster_name', rel_name)
            cur_vlass_res.insert(1, 'nominal_vlass_coverage', nominal_coverage)
            # Calculate the separations from the input BCG coordinate
            cur_seps = SkyCoord(cur_vlass_res[['RAJ2000', 'DEJ2000']].values, unit='deg').separation(cur_coord).to('arcsec')
            cur_vlass_res['arcsec_sep_from_bcg'] = cur_seps.value

            # Sort so that the closest match appears highest in the table
            cur_vlass_res.sort_values('arcsec_sep_from_bcg', inplace=True)

            # Making the dataframe into a list that can be appended to our wider list for all clusters (it'll get 
            #  made back into a dataframe at the end) - yes it isn't very elegant but who cares
            bcg_cand_vlass += cur_vlass_res.values.tolist()
    
            # Stores the columns to help us reconstruct our dataframe later
            if cols is None:
                cols = cur_vlass_res.columns.values
        else:
            bcg_cand_vlass.append([rel_name, nominal_coverage])

    # Getting this far and cols still being None means that there were no matches from the catalog - still
    #  want some column names though, as we will have included cluster names and nominal coverage
    # This flag will help later when we're parsing the dataframe - we won't bother grouping the dataframe to find
    #  out how many matches there were for each cluster's candidate
    no_results = False
    if cols is None:
        # If we get here, there were no matches at all
        no_results = True
        cols = ['cluster_name', 'nominal_vlass_coverage']
        
    # The whole output dataframe of our search - this includes null results, and whether there is nominally
    #  meant to be coverage from the catalog for that BCG candidate
    bcg_cand_vlass = pd.DataFrame(bcg_cand_vlass, columns=cols)

    # We have to check if there were any results at all, as we can add a 'CompName' column full
    #  of null results to trick the latter stages into working normally
    if no_results:
        bcg_cand_vlass['CompName'] = None 
           
    # Summarising the number of matches for each candidate
    bcg_cand_vlass_nummatches = bcg_cand_vlass.groupby('cluster_name')['CompName'].describe()['count']
        
    # And fishing out those with no matches - surely there is a more elegant way of doing this
    bcg_cand_vlass_nomatch_incov_clnames = bcg_cand_vlass_nummatches[bcg_cand_vlass_nummatches == 0].index.values
    any_cov = bcg_cand_vlass[['cluster_name', 
                              'nominal_vlass_coverage']].set_index('cluster_name').loc[bcg_cand_vlass_nomatch_incov_clnames]
    bcg_cand_vlass_nomatch_incov_clnames = bcg_cand_vlass_nomatch_incov_clnames[any_cov.values.flatten()]
    
    return bcg_cand_vlass, bcg_cand_vlass_nummatches, bcg_cand_vlass_nomatch_incov_clnames


def galex_search(coords_to_search):
    bcg_cand_galex = []

    # This will be replaced with the columns pulled from the catalog (plus some extras) later on
    cols = None
    
    # We iterate because even though .query_region is perfectly happy taking a set of coordinates, the returned table 
    #  does not associate particular catalog entries with a particular coordinate that we passed. We do want that information, and
    #  while we could reconstruct it after a query with all coordinates passed, we are going to do it by querying on a galaxy-by-galaxy
    #  basis
    for cur_coord_ind, cur_coord in enumerate(coords_to_search):
        rel_name = bcg_cand_samp['cluster_name'].values[cur_coord_ind]
        
        # Possible there will be NaN coordinates, need to account for that
        if np.isnan(cur_coord.ra.value):
            continue
    
        # Performs the actual search through the catalog
        cur_galex_res = galex_cat.query_region(cur_coord, radius=GALEX_SEARCH_RAD)
        
        # Nominally every source should be covered by this catalog, as it was an all-sky survey. In reality
        #  I'm sure there are missing chunks, but identifying them is probably too much effort for this application
        nominal_coverage = True
    
        # Checking and storing results
        if len(cur_galex_res) > 0 and 'II/335/galex_ais' in list(cur_galex_res.keys()):
            # The key we pass at the end is to retrieve a specific table of matches (some catalogs have multiple tables)
            cur_galex_res = cur_galex_res['II/335/galex_ais'].to_pandas()
            # Make sure the cluster name is in the dataframe
            cur_galex_res.insert(0, 'cluster_name', rel_name)            
            cur_galex_res.insert(1, 'nominal_galex_coverage', nominal_coverage)
            
            # Calculate the separations from the input BCG coordinate
            cur_seps = SkyCoord(cur_galex_res[['RAJ2000', 'DEJ2000']].values, unit='deg').separation(cur_coord).to('arcsec')
            cur_galex_res['arcsec_sep_from_bcg'] = cur_seps.value

            # Sort so that the closest match appears highest in the table
            cur_galex_res.sort_values('arcsec_sep_from_bcg', inplace=True)

            # Making the dataframe into a list that can be appended to our wider list for all clusters (it'll get 
            #  made back into a dataframe at the end) - yes it isn't very elegant but who cares
            bcg_cand_galex += cur_galex_res.values.tolist()
    
            # Stores the columns to help us reconstruct our dataframe later
            if cols is None:
                cols = cur_galex_res.columns.values
        else:
            bcg_cand_galex.append([rel_name, nominal_coverage])

    # Getting this far and cols still being None means that there were no matches from the catalog - still
    #  want some column names though, as we will have included cluster names and nominal coverage
    # This flag will help later when we're parsing the dataframe - we won't bother grouping the dataframe to find
    #  out how many matches there were for each cluster's candidate
    no_results = False
    if cols is None:
        # If we get here, there were no matches at all
        no_results = True
        cols = ['cluster_name', 'nominal_galex_coverage']
        
    # The whole output dataframe of our search - this includes null results, and whether there is nominally
    #  meant to be coverage from the catalog for that BCG candidate
    bcg_cand_galex = pd.DataFrame(bcg_cand_galex, columns=cols)

    # We have to check if there were any results at all, as we can add a 'CompName' column full
    #  of null results to trick the latter stages into working normally
    if no_results:
        bcg_cand_galex['CompName'] = None 
           
    # Summarising the number of matches for each candidate
    bcg_cand_galex_nummatches = bcg_cand_galex.groupby('cluster_name')['Name'].describe()['count']
        
    # And fishing out those with no matches - surely there is a more elegant way of doing this
    bcg_cand_galex_nomatch_incov_clnames = bcg_cand_galex_nummatches[bcg_cand_galex_nummatches == 0].index.values
    any_cov = bcg_cand_galex[['cluster_name', 
                              'nominal_galex_coverage']].set_index('cluster_name').loc[bcg_cand_galex_nomatch_incov_clnames]
    bcg_cand_galex_nomatch_incov_clnames = bcg_cand_galex_nomatch_incov_clnames[any_cov.values.flatten()]
    
    return bcg_cand_galex, bcg_cand_galex_nummatches, bcg_cand_galex_nomatch_incov_clnames

## Loading X-LoVoCCS BCG candidate samples

In [5]:
bcg_cand_samp = pd.read_csv("outputs/bcg_output_sample.csv")
# Allow all rows to be displayed
pd.set_option('display.max_rows', None)
# The 'display' function means we can have a nice rendered version of the DF without it being the 
#  very last thing in the notebook
display(bcg_cand_samp)
# Resetting the number of rows displayed
pd.set_option('display.max_rows', 15)

,cluster_name,no_bcg_cand,BCG1_desi-ls_ra,BCG1_desi-ls_dec,BCG2_desi-ls_ra,BCG2_desi-ls_dec,BCG3_desi-ls_ra,BCG3_desi-ls_dec,BCG4_desi-ls_ra,BCG4_desi-ls_dec,BCG1_lovoccs_ra,BCG1_lovoccs_dec,BCG2_lovoccs_ra,BCG2_lovoccs_dec
0,LoVoCCS-1,False,227.733824,5.744883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LoVoCCS-2,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.740836,13.582646,NaN,NaN
2,LoVoCCS-4A,False,10.460194,-9.302871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LoVoCCS-4B,False,10.429048,-9.439317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LoVoCCS-5,False,303.113338,-56.826500,302.710346,-56.673695,303.506670,-57.027568,303.494070,-57.039226,NaN,NaN,NaN,NaN
5,LoVoCCS-7,False,330.470382,-59.945214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LoVoCCS-9,False,67.802961,-61.453626,67.414206,-61.176134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LoVoCCS-10,False,194.843512,-4.196002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LoVoCCS-11,False,137.134448,-9.629780,137.329532,-9.698835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,LoVoCCS-12,False,206.867783,-32.864949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Using the sample dataframe to setup some astropy coordinate objects:

In [6]:
# Two clusters don't have DESI-LS defined BCG coords (A401 and A399), so we have to do this silliness
bcg1_fixed = [ccoord if ~np.isnan(ccoord).any() else bcg_cand_samp[['BCG1_lovoccs_ra', 'BCG1_lovoccs_dec']].values[ccoord_ind] 
              for ccoord_ind, ccoord in enumerate(bcg_cand_samp[['BCG1_desi-ls_ra', 'BCG1_desi-ls_dec']].values)]
bcg1_cand_coords = SkyCoord(bcg1_fixed, unit='deg')

# Same silliness as above
bcg2_fixed = [ccoord if ~np.isnan(ccoord).any() else bcg_cand_samp[['BCG2_lovoccs_ra', 'BCG2_lovoccs_dec']].values[ccoord_ind] 
              for ccoord_ind, ccoord in enumerate(bcg_cand_samp[['BCG2_desi-ls_ra', 'BCG2_desi-ls_dec']].values)]
bcg2_cand_coords = SkyCoord(bcg2_fixed, unit='deg')

# The BCG3 and 4 columns don't have LoVoCCS equivalents
bcg3_cand_coords = SkyCoord(bcg_cand_samp[['BCG3_desi-ls_ra', 'BCG3_desi-ls_dec']].values, unit='deg')
bcg4_cand_coords = SkyCoord(bcg_cand_samp[['BCG4_desi-ls_ra', 'BCG4_desi-ls_dec']].values, unit='deg')

## Loading VLASS and GALEX catalogs from VizieR

### VLASS QL Ep. 1

In [7]:
vlass_cat = Vizier(catalog='J/ApJS/255/30', columns=['**'])
vlass_cat.get_catalog_metadata()[['title', 'authors', 'origin_article', 'webpage', 'created', 'updated', 'waveband', 'doi']]

title,authors,origin_article,webpage,created,updated,waveband,doi
object,object,object,object,object,object,object,object
"VLASS QL Ep.1 Catalog, CIRADA version","Gordon Y.A.; Boyce M.M.; O'Dea C.P.; Rudnick L.; Andernach H.,Vantyghem A.N.; Baum S.A.; Bui J.-P.; Dionyssiou M.; Safi-Harb S.; Sander I.",2021ApJS..255...30G,https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJS/255/30,2021-08-18T08:26:04,2025-05-19T09:50:00,radio,doi:10.26093/cds/vizier.22550030


### GALEX UV catalog AIS GR6+7

In [8]:
galex_cat = Vizier(catalog='II/335', columns=['**'])
galex_cat.get_catalog_metadata()[['title', 'authors', 'origin_article', 'webpage', 'created', 'updated', 'waveband', 'doi']]

title,authors,origin_article,webpage,created,updated,waveband,doi
object,object,object,object,object,object,object,object
Revised catalog of GALEX UV sources (GUVcat_AIS GR6+7),Bianchi L.; Shiao B.; Thilker D.,2017ApJS..230...24B,https://cdsarc.cds.unistra.fr/viz-bin/cat/II/335,2014-11-26T09:51:14,2025-05-19T09:50:00,uv,--


## Searching around BCG coordinates

The Vizier object in AstroQuery doesn't seem to be able to associate specific identified entries in catalogs with a particular input coordinate - as such we iterate through our candidates and search for and store nearby entries in the catalogs individually

### VLASS

#### BCG1 candidates

In [9]:
bcg1_cand_vlass, bcg1_cand_vlass_nummatches, bcg1_cand_vlass_nomatch_clnames = vlass_search(bcg1_cand_coords)

The main output table from the matching process:

In [10]:
bcg1_cand_vlass.to_csv(out_dir + "bcg1_cands_vlass_searchrad{r}.csv".format(r=str(VLASS_SEARCH_RAD).replace(' ', '')), index=False)
bcg1_cand_vlass

,cluster_name,nominal_vlass_coverage,_r,recno,CompName,CompId,IslId,RAJ2000,DEJ2000,e_RAJ2000,...,PeakToRing,DupFlag,QualFlag,NNdist,BMaj,BMin,BPA,MainSample,QLcutout,arcsec_sep_from_bcg
0,LoVoCCS-1,True,5.188,2145337.0,VLASS1QLCIR J151056.30+054437.1,36.0,64.0,227.734597,5.743662,0.000007,...,1.1153,0.0,0.0,10.48,2.73,2.69,13.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,5.191896
1,LoVoCCS-1,True,5.558,2145311.0,VLASS1QLCIR J151055.87+054445.7,35.0,64.0,227.732813,5.746051,0.000007,...,0.9455,0.0,0.0,10.71,2.73,2.69,13.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,5.551222
2,LoVoCCS-2,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,LoVoCCS-4A,True,0.813,98098.0,VLASS1QLCIR J004150.48-091810.8,50.0,55.0,10.460373,-9.303017,0.000003,...,29.1177,0.0,0.0,19.87,3.75,2.29,21.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,0.824652
4,LoVoCCS-4B,True,0.633,97825.0,VLASS1QLCIR J004143.01-092621.4,60.0,64.0,10.429229,-9.439285,0.000007,...,15.6389,0.0,0.0,111.47,3.75,2.29,21.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,0.653081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,LoVoCCS-123,True,9.589,1819221.0,VLASS1QLCIR J125440.32-291337.8,65.0,83.0,193.668002,-29.227189,0.000049,...,0.0908,0.0,0.0,6.07,2.89,1.79,44.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,9.558508
79,LoVoCCS-131,True,1.206,1294508.0,VLASS1QLCIR J091035.90-103456.8,27.0,25.0,137.649608,-10.582471,0.000002,...,0.8930,0.0,4.0,8.40,3.50,2.16,14.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,1.227382
80,LoVoCCS-131,True,8.729,1294522.0,VLASS1QLCIR J091036.35-103450.0,23.0,24.0,137.651477,-10.580567,0.000018,...,1.3179,0.0,0.0,8.23,3.50,2.16,14.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,8.734326
81,LoVoCCS-131,True,9.094,1294485.0,VLASS1QLCIR J091035.43-103501.5,25.0,25.0,137.647636,-10.583769,0.000031,...,1.8307,0.0,0.0,8.40,3.50,2.16,14.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,9.088437


In [11]:
bcg1_cand_vlass[bcg1_cand_vlass['cluster_name'] == 'LoVoCCS-123']

,cluster_name,nominal_vlass_coverage,_r,recno,CompName,CompId,IslId,RAJ2000,DEJ2000,e_RAJ2000,...,PeakToRing,DupFlag,QualFlag,NNdist,BMaj,BMin,BPA,MainSample,QLcutout,arcsec_sep_from_bcg
74,LoVoCCS-123,True,3.363,1819250.0,VLASS1QLCIR J125441.08-291342.1,64.0,83.0,193.671208,-29.228384,0.000006,...,2.4988,0.0,0.0,5.77,2.89,1.79,44.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,3.371597
75,LoVoCCS-123,True,6.195,1819234.0,VLASS1QLCIR J125440.64-291342.1,66.0,83.0,193.669370,-29.228380,0.000107,...,0.9313,0.0,0.0,5.77,2.89,1.79,44.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,6.171630
76,LoVoCCS-123,True,8.407,1819232.0,VLASS1QLCIR J125440.63-291332.4,70.0,83.0,193.669306,-29.225681,0.000091,...,0.0657,0.0,0.0,6.80,2.89,1.79,44.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,8.383832
77,LoVoCCS-123,True,9.378,1819268.0,VLASS1QLCIR J125441.63-291344.3,68.0,83.0,193.673465,-29.228978,0.000041,...,0.1991,0.0,0.0,7.41,2.89,1.79,44.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,9.405257
78,LoVoCCS-123,True,9.589,1819221.0,VLASS1QLCIR J125440.32-291337.8,65.0,83.0,193.668002,-29.227189,0.000049,...,0.0908,0.0,0.0,6.07,2.89,1.79,44.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,9.558508


Here we look at the number of matches found for this BCG candidate for each cluster:

In [12]:
pd.set_option('display.max_rows', None)
display(bcg1_cand_vlass_nummatches.sort_values(ascending=False))
# Resetting the number of rows displayed
pd.set_option('display.max_rows', 15)

cluster_name
LoVoCCS-123    5
LoVoCCS-48A    4
LoVoCCS-67     3
LoVoCCS-131    3
LoVoCCS-1      2
LoVoCCS-90     2
LoVoCCS-61     2
LoVoCCS-41C    2
LoVoCCS-99     2
LoVoCCS-49     1
LoVoCCS-31     1
LoVoCCS-119    1
LoVoCCS-4B     1
LoVoCCS-4A     1
LoVoCCS-122    1
LoVoCCS-48B    1
LoVoCCS-12     1
LoVoCCS-46C    1
LoVoCCS-10     1
LoVoCCS-66A    1
LoVoCCS-66B    1
LoVoCCS-39     1
LoVoCCS-60A    1
LoVoCCS-75     1
LoVoCCS-30     1
LoVoCCS-98     1
LoVoCCS-15     1
LoVoCCS-21     1
LoVoCCS-22     1
LoVoCCS-76     0
LoVoCCS-63     0
LoVoCCS-94     0
LoVoCCS-93B    0
LoVoCCS-93A    0
LoVoCCS-65     0
LoVoCCS-108    0
LoVoCCS-9      0
LoVoCCS-89     0
LoVoCCS-11     0
LoVoCCS-7      0
LoVoCCS-74     0
LoVoCCS-85     0
LoVoCCS-80     0
LoVoCCS-60B    0
LoVoCCS-13     0
LoVoCCS-58     0
LoVoCCS-55     0
LoVoCCS-134    0
LoVoCCS-14     0
LoVoCCS-18     0
LoVoCCS-2      0
LoVoCCS-24     0
LoVoCCS-26     0
LoVoCCS-27     0
LoVoCCS-28     0
LoVoCCS-29     0
LoVoCCS-33     0
LoVoCCS-35     0
L

Finally, list the clusters for which there is no match, and nominally there should be coverage:

In [13]:
bcg1_cand_vlass_nomatch_clnames

array(['LoVoCCS-108', 'LoVoCCS-11', 'LoVoCCS-14', 'LoVoCCS-18',
       'LoVoCCS-2', 'LoVoCCS-26', 'LoVoCCS-28', 'LoVoCCS-29',
       'LoVoCCS-33', 'LoVoCCS-41A', 'LoVoCCS-41B', 'LoVoCCS-46A',
       'LoVoCCS-46B', 'LoVoCCS-55', 'LoVoCCS-58', 'LoVoCCS-60B',
       'LoVoCCS-63', 'LoVoCCS-65', 'LoVoCCS-76', 'LoVoCCS-85',
       'LoVoCCS-89', 'LoVoCCS-94'], dtype=object)

#### BCG2 candidates

In [14]:
bcg2_cand_vlass, bcg2_cand_vlass_nummatches, bcg2_cand_vlass_nomatch_clnames = vlass_search(bcg2_cand_coords)

The main output table from the matching process:

In [15]:
bcg2_cand_vlass.to_csv(out_dir + "bcg2_cands_vlass_searchrad{r}.csv".format(r=str(VLASS_SEARCH_RAD).replace(' ', '')), index=False)
bcg2_cand_vlass

,cluster_name,nominal_vlass_coverage,_r,recno,CompName,CompId,IslId,RAJ2000,DEJ2000,e_RAJ2000,...,PeakToRing,DupFlag,QualFlag,NNdist,BMaj,BMin,BPA,MainSample,QLcutout,arcsec_sep_from_bcg
0,LoVoCCS-5,False,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
1,LoVoCCS-9,False,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2,LoVoCCS-11,True,4.201,1291547.0,VLASS1QLCIR J090919.27-094159.0,46.0,50.0,137.330312,-9.699724,0.000062,...,4.2743,0.0,0.0,83.07,3.60,2.23,20.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,4.232031
3,LoVoCCS-13,False,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
4,LoVoCCS-14,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,LoVoCCS-61,True,6.258,3161187.0,VLASS1QLCIR J222602.31+172242.1,15.0,17.0,336.509638,17.378368,0.000071,...,2.8230,0.0,0.0,44.44,2.52,2.38,-75.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,6.256378
23,LoVoCCS-80,False,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
24,LoVoCCS-108,True,1.379,1641310.0,VLASS1QLCIR J113924.51-332645.4,-3.0,4.0,174.852163,-33.445946,0.000000,...,1.4361,0.0,4.0,NaN,3.08,1.99,48.0,0.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,1.402866
25,LoVoCCS-131,True,1.191,1295306.0,VLASS1QLCIR J091057.99-103352.4,16.0,18.0,137.741650,-10.564576,0.000007,...,9.1361,0.0,0.0,318.37,3.50,2.16,14.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,1.220157


Here we look at the number of matches found for this BCG candidate for each cluster:

In [16]:
pd.set_option('display.max_rows', None)
display(bcg2_cand_vlass_nummatches.sort_values(ascending=False))
# Resetting the number of rows displayed
pd.set_option('display.max_rows', 15)

cluster_name
LoVoCCS-41C    2
LoVoCCS-55     2
LoVoCCS-108    1
LoVoCCS-131    1
LoVoCCS-61     1
LoVoCCS-21     1
LoVoCCS-22     1
LoVoCCS-58     1
LoVoCCS-30     1
LoVoCCS-11     1
LoVoCCS-80     0
LoVoCCS-60A    0
LoVoCCS-5      0
LoVoCCS-45     0
LoVoCCS-42     0
LoVoCCS-31     0
LoVoCCS-39     0
LoVoCCS-33     0
LoVoCCS-28     0
LoVoCCS-27     0
LoVoCCS-24     0
LoVoCCS-14     0
LoVoCCS-134    0
LoVoCCS-13     0
LoVoCCS-9      0
Name: count, dtype: object

Finally, list the clusters for which there is no match, and nominally there should be coverage:

In [17]:
bcg2_cand_vlass_nomatch_clnames

array(['LoVoCCS-14', 'LoVoCCS-28', 'LoVoCCS-31', 'LoVoCCS-33',
       'LoVoCCS-39', 'LoVoCCS-60A'], dtype=object)

#### BCG3 candidates

In [18]:
bcg3_cand_vlass, bcg3_cand_vlass_nummatches, bcg3_cand_vlass_nomatch_clnames = vlass_search(bcg3_cand_coords)

The main output table from the matching process:

In [19]:
bcg3_cand_vlass.to_csv(out_dir + "bcg3_cands_vlass_searchrad{r}.csv".format(r=str(VLASS_SEARCH_RAD).replace(' ', '')), index=False)
bcg3_cand_vlass

,cluster_name,nominal_vlass_coverage,_r,recno,CompName,CompId,IslId,RAJ2000,DEJ2000,e_RAJ2000,...,PeakToRing,DupFlag,QualFlag,NNdist,BMaj,BMin,BPA,MainSample,QLcutout,arcsec_sep_from_bcg
0,LoVoCCS-5,False,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
1,LoVoCCS-22,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2,LoVoCCS-27,False,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,LoVoCCS-28,True,5.054,1734747.0,VLASS1QLCIR J121731.38+033700.5,46.0,45.0,184.380759,3.616822,0.000072,...,0.5657,0.0,0.0,10.46,2.89,2.18,27.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,5.048602
4,LoVoCCS-28,True,5.508,1734741.0,VLASS1QLCIR J121731.22+033650.3,47.0,45.0,184.380108,3.613991,0.000047,...,0.8165,0.0,0.0,10.46,2.89,2.18,27.0,1.0,https://ws.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/ca...,5.507720
5,LoVoCCS-39,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
6,LoVoCCS-55,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
7,LoVoCCS-58,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
8,LoVoCCS-134,False,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


Here we look at the number of matches found for this BCG candidate for each cluster:

In [20]:
pd.set_option('display.max_rows', None)
display(bcg3_cand_vlass_nummatches.sort_values(ascending=False))
# Resetting the number of rows displayed
pd.set_option('display.max_rows', 15)

cluster_name
LoVoCCS-28     2
LoVoCCS-134    0
LoVoCCS-22     0
LoVoCCS-27     0
LoVoCCS-39     0
LoVoCCS-5      0
LoVoCCS-55     0
LoVoCCS-58     0
Name: count, dtype: object

Finally, list the clusters for which there is no match, and nominally there should be coverage:

In [21]:
bcg3_cand_vlass_nomatch_clnames

array(['LoVoCCS-22', 'LoVoCCS-39', 'LoVoCCS-55', 'LoVoCCS-58'],
      dtype=object)

#### BCG4 candidates

**We note that there are no matches for BCG4 candidates at the time of running (21st May 2025)**

In [22]:
bcg4_cand_vlass, bcg4_cand_vlass_nummatches, bcg4_cand_vlass_nomatch_clnames = vlass_search(bcg4_cand_coords)

The main output table from the matching process:

In [23]:
bcg4_cand_vlass.to_csv(out_dir + "bcg4_cands_vlass_searchrad{r}.csv".format(r=str(VLASS_SEARCH_RAD).replace(' ', '')), index=False)
bcg4_cand_vlass

,cluster_name,nominal_vlass_coverage,CompName
0,LoVoCCS-5,False,None
1,LoVoCCS-39,True,None
2,LoVoCCS-55,True,None


Here we look at the number of matches found for this BCG candidate for each cluster:

In [24]:
pd.set_option('display.max_rows', None)
display(bcg4_cand_vlass_nummatches.sort_values(ascending=False))
# Resetting the number of rows displayed
pd.set_option('display.max_rows', 15)

cluster_name
LoVoCCS-39    0
LoVoCCS-5     0
LoVoCCS-55    0
Name: count, dtype: object

Finally, list the clusters for which there is no match, and nominally there should be coverage:

In [25]:
bcg4_cand_vlass_nomatch_clnames

array(['LoVoCCS-39', 'LoVoCCS-55'], dtype=object)

### GALEX

#### BCG1 candidates

In [26]:
bcg1_cand_galex, bcg1_cand_galex_nummatches, bcg1_cand_galex_nomatch_clnames = galex_search(bcg1_cand_coords)

Main output table from the matching process:

In [27]:
bcg1_cand_galex.to_csv(out_dir + "bcg1_cands_galex_searchrad{r}.csv".format(r=str(GALEX_SEARCH_RAD).replace(' ', '')), index=False)
bcg1_cand_galex

,cluster_name,nominal_galex_coverage,_r,RAJ2000,DEJ2000,Name,objid,phID,Cat,RAfdeg,...,primid,groupid,Gd,Nd,primidd,groupidd,groupTot,OName,Size,arcsec_sep_from_bcg
0,LoVoCCS-1,True,2.657,227.733100,5.744756,GALEX J151055.9+054441,6.382773e+18,6.382773e+18,AIS,227.849966,...,6.382773e+18,6382772910359052602,0.0,1.0,6.382773e+18,6382772910359052602,6382772910359052602,GA:IC1101,71.80,2.634205
1,LoVoCCS-2,True,2.436,44.740156,13.582539,GALEX J025857.6+133457,6.377073e+18,6.377073e+18,AIS,44.713663,...,6.377073e+18,6377073035636116665,0.0,1.0,6.377073e+18,6377073035636116665,6377073035636116665,GA:UGC02450,60.83,2.409283
2,LoVoCCS-4A,True,4.393,10.460541,-9.304042,GALEX J004150.5-091814,6.380451e+18,6.380451e+18,AIS,10.666415,...,6.380451e+18,6380450727842549258,0.0,1.0,6.380451e+18,6380450727842549258,6380450727842549258,GA:PGC002501,87.93,4.392614
3,LoVoCCS-4B,True,1.235,10.429076,-9.438973,GALEX J004142.9-092620,6.380451e+18,6.380451e+18,AIS,10.666415,...,6.380451e+18,6380450727842548858,0.0,1.0,6.380451e+18,6380450727842548858,6380450727842548858,GA:PGC002480,67.17,1.241146
4,LoVoCCS-5,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,LoVoCCS-122,True,1.867,156.994922,-6.798815,GALEX J102758.7-064755,6.382034e+18,6.382034e+18,AIS,157.233207,...,6.382034e+18,6382034091158538613,0.0,1.0,6.382034e+18,6382034091158538613,6382034091158538613,N,NaN,1.889867
66,LoVoCCS-123,True,0.917,193.671284,-29.227598,GALEX J125441.1-291339,6.388191e+18,6.388191e+18,AIS,193.815444,...,6.388191e+18,6388191378822664905,0.0,1.0,6.388191e+18,6388191378822664905,6388191378822664905,GA:ESO443-007,61.68,0.943266
67,LoVoCCS-131,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN
68,LoVoCCS-134,True,2.060,329.606344,-60.391760,GALEX J215825.5-602330,6.383758e+18,6.383758e+18,AIS,329.782211,...,6.383758e+18,6383758094252376625,0.0,1.0,6.383758e+18,6383758094252376625,6383758094252376625,N,NaN,2.040800


Here we look at the number of matches found for this BCG candidate for each cluster:

In [28]:
pd.set_option('display.max_rows', None)
display(bcg1_cand_galex_nummatches.sort_values(ascending=False))
# Resetting the number of rows displayed
pd.set_option('display.max_rows', 15)

cluster_name
LoVoCCS-48B    2
LoVoCCS-21     2
LoVoCCS-134    2
LoVoCCS-1      1
LoVoCCS-67     1
LoVoCCS-41B    1
LoVoCCS-42     1
LoVoCCS-49     1
LoVoCCS-4A     1
LoVoCCS-4B     1
LoVoCCS-60A    1
LoVoCCS-65     1
LoVoCCS-7      1
LoVoCCS-39     1
LoVoCCS-74     1
LoVoCCS-75     1
LoVoCCS-76     1
LoVoCCS-80     1
LoVoCCS-85     1
LoVoCCS-89     1
LoVoCCS-9      1
LoVoCCS-90     1
LoVoCCS-98     1
LoVoCCS-41A    1
LoVoCCS-99     1
LoVoCCS-121    1
LoVoCCS-119    1
LoVoCCS-2      1
LoVoCCS-15     1
LoVoCCS-14     1
LoVoCCS-13     1
LoVoCCS-123    1
LoVoCCS-122    1
LoVoCCS-18     1
LoVoCCS-11     1
LoVoCCS-28     1
LoVoCCS-29     1
LoVoCCS-12     0
LoVoCCS-63     0
LoVoCCS-93A    0
LoVoCCS-93B    0
LoVoCCS-131    0
LoVoCCS-94     0
LoVoCCS-66B    0
LoVoCCS-66A    0
LoVoCCS-108    0
LoVoCCS-33     0
LoVoCCS-60B    0
LoVoCCS-61     0
LoVoCCS-48A    0
LoVoCCS-31     0
LoVoCCS-30     0
LoVoCCS-41C    0
LoVoCCS-45     0
LoVoCCS-10     0
LoVoCCS-46B    0
LoVoCCS-46C    0
LoVoCCS-27     0
L

Finally, list the clusters for which there is no match, and nominally there should be coverage:

In [29]:
bcg1_cand_galex_nomatch_clnames

array(['LoVoCCS-10', 'LoVoCCS-108', 'LoVoCCS-12', 'LoVoCCS-131',
       'LoVoCCS-22', 'LoVoCCS-24', 'LoVoCCS-26', 'LoVoCCS-27',
       'LoVoCCS-30', 'LoVoCCS-31', 'LoVoCCS-33', 'LoVoCCS-35',
       'LoVoCCS-41C', 'LoVoCCS-45', 'LoVoCCS-46A', 'LoVoCCS-46B',
       'LoVoCCS-46C', 'LoVoCCS-48A', 'LoVoCCS-5', 'LoVoCCS-51',
       'LoVoCCS-55', 'LoVoCCS-58', 'LoVoCCS-60B', 'LoVoCCS-61',
       'LoVoCCS-63', 'LoVoCCS-66A', 'LoVoCCS-66B', 'LoVoCCS-93A',
       'LoVoCCS-93B', 'LoVoCCS-94'], dtype=object)

#### BCG2 candidates

In [30]:
bcg2_cand_galex, bcg2_cand_galex_nummatches, bcg2_cand_galex_nomatch_clnames = galex_search(bcg2_cand_coords)

Main output table from the matching process:

In [31]:
bcg2_cand_galex.to_csv(out_dir + "bcg2_cands_galex_searchrad{r}.csv".format(r=str(GALEX_SEARCH_RAD).replace(' ', '')), index=False)
bcg2_cand_galex

,cluster_name,nominal_galex_coverage,_r,RAJ2000,DEJ2000,Name,objid,phID,Cat,RAfdeg,...,primid,groupid,Gd,Nd,primidd,groupidd,groupTot,OName,Size,arcsec_sep_from_bcg
0,LoVoCCS-5,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN
1,LoVoCCS-9,True,1.091,67.414796,-61.176240,GALEX J042939.5-611034,6.385693e+18,6.385693e+18,AIS,67.812027,...,6.385693e+18,6385693257263746233,0.0,1.0,6.385693e+18,6385693257263746233,6385693257263746233,N,NaN,1.092973
2,LoVoCCS-11,True,5.193,137.330583,-9.699851,GALEX J090919.3-094159,6.378340e+18,6.378340e+18,AIS,137.418690,...,6.378340e+18,6378339691287022393,0.0,1.0,6.378340e+18,6378339691287022393,6378339691287022393,N,NaN,5.224076
3,LoVoCCS-13,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN
4,LoVoCCS-14,True,1.950,44.535837,13.073483,GALEX J025808.6+130424,6.377073e+18,6.377073e+18,AIS,44.713663,...,6.377073e+18,6377073035636114844,0.0,1.0,6.377073e+18,6377073035636114844,6377073035636114844,N,NaN,1.941304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,LoVoCCS-80,True,2.002,52.128003,-55.704982,GALEX J032830.7-554217,6.385869e+18,6.385869e+18,AIS,52.786776,...,6.385869e+18,6385869039537751512,0.0,1.0,6.385869e+18,6385869039537751512,6385869039537751512,N,NaN,2.011742
22,LoVoCCS-108,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN
23,LoVoCCS-131,True,4.299,137.742573,-10.564814,GALEX J091058.2-103353,6.378340e+18,6.378340e+18,AIS,137.901547,...,6.378340e+18,6378339702022344005,0.0,1.0,6.378340e+18,6378339702022344005,6378339702022344005,N,NaN,4.329078
24,LoVoCCS-134,True,0.258,329.608322,-60.426015,GALEX J215825.9-602533,6.383758e+18,6.383758e+18,AIS,329.932286,...,6.383758e+18,6383758098545251781,0.0,1.0,6.383758e+18,6383758098545251781,6383758098545251781,N,NaN,0.271633


Here we look at the number of matches found for this BCG candidate for each cluster:

In [32]:
pd.set_option('display.max_rows', None)
display(bcg2_cand_galex_nummatches.sort_values(ascending=False))
# Resetting the number of rows displayed
pd.set_option('display.max_rows', 15)

cluster_name
LoVoCCS-134    2
LoVoCCS-9      1
LoVoCCS-28     1
LoVoCCS-41C    1
LoVoCCS-39     1
LoVoCCS-11     1
LoVoCCS-60A    1
LoVoCCS-21     1
LoVoCCS-14     1
LoVoCCS-131    1
LoVoCCS-80     1
LoVoCCS-42     1
LoVoCCS-61     0
LoVoCCS-45     0
LoVoCCS-58     0
LoVoCCS-55     0
LoVoCCS-5      0
LoVoCCS-108    0
LoVoCCS-33     0
LoVoCCS-30     0
LoVoCCS-27     0
LoVoCCS-24     0
LoVoCCS-22     0
LoVoCCS-13     0
LoVoCCS-31     0
Name: count, dtype: object

Finally, list the clusters for which there is no match, and nominally there should be coverage:

In [33]:
bcg2_cand_galex_nomatch_clnames

array(['LoVoCCS-108', 'LoVoCCS-13', 'LoVoCCS-22', 'LoVoCCS-24',
       'LoVoCCS-27', 'LoVoCCS-30', 'LoVoCCS-31', 'LoVoCCS-33',
       'LoVoCCS-45', 'LoVoCCS-5', 'LoVoCCS-55', 'LoVoCCS-58',
       'LoVoCCS-61'], dtype=object)

#### BCG3 candidates

In [34]:
bcg3_cand_galex, bcg3_cand_galex_nummatches, bcg3_cand_galex_nomatch_clnames = galex_search(bcg3_cand_coords)

Main output table from the matching process:

In [35]:
bcg3_cand_galex.to_csv(out_dir + "bcg3_cands_galex_searchrad{r}.csv".format(r=str(GALEX_SEARCH_RAD).replace(' ', '')), index=False)
bcg3_cand_galex

,cluster_name,nominal_galex_coverage,_r,RAJ2000,DEJ2000,Name,objid,phID,Cat,RAfdeg,...,primid,groupid,Gd,Nd,primidd,groupidd,groupTot,OName,Size,arcsec_sep_from_bcg
0,LoVoCCS-5,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN
1,LoVoCCS-22,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN
2,LoVoCCS-27,True,1.559,55.873913,-53.692098,GALEX J034329.7-534131,6.385869e+18,6.385869e+18,AIS,56.708925,...,6.385869e+18,6385869173757577461,0.0,1.0,6.385869e+18,6385869173757577461,6385869173757577461,GA:PGC013679,64.29,1.564297
3,LoVoCCS-28,True,1.641,184.381076,3.615258,GALEX J121731.4+033654,6.378938e+18,6.378938e+18,AIS,184.057116,...,6.378938e+18,6378937813799273711,0.0,1.0,6.378938e+18,6378937813799273711,6378937813799273711,N,NaN,1.663258
4,LoVoCCS-39,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN
5,LoVoCCS-55,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN
6,LoVoCCS-58,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN
7,LoVoCCS-134,True,1.250,329.539613,-60.244931,GALEX J215809.5-601441,6.383758e+18,6.383758e+18,AIS,329.782211,...,6.383758e+18,6383758094252377266,0.0,1.0,6.383758e+18,6383758094252377266,6383758094252377266,N,NaN,1.260374


Here we look at the number of matches found for this BCG candidate for each cluster:

In [36]:
pd.set_option('display.max_rows', None)
display(bcg3_cand_galex_nummatches.sort_values(ascending=False))
# Resetting the number of rows displayed
pd.set_option('display.max_rows', 15)

cluster_name
LoVoCCS-134    1
LoVoCCS-27     1
LoVoCCS-28     1
LoVoCCS-22     0
LoVoCCS-39     0
LoVoCCS-5      0
LoVoCCS-55     0
LoVoCCS-58     0
Name: count, dtype: object

Finally, list the clusters for which there is no match, and nominally there should be coverage:

In [37]:
bcg3_cand_galex_nomatch_clnames

array(['LoVoCCS-22', 'LoVoCCS-39', 'LoVoCCS-5', 'LoVoCCS-55',
       'LoVoCCS-58'], dtype=object)

#### BCG4 candidates

In [38]:
bcg4_cand_galex, bcg4_cand_galex_nummatches, bcg4_cand_galex_nomatch_clnames = galex_search(bcg4_cand_coords)

Main output table from the matching process:

In [39]:
bcg4_cand_galex.to_csv(out_dir + "bcg4_cands_galex_searchrad{r}.csv".format(r=str(GALEX_SEARCH_RAD).replace(' ', '')), index=False)
bcg4_cand_galex

,cluster_name,nominal_galex_coverage,_r,RAJ2000,DEJ2000,Name,objid,phID,Cat,RAfdeg,...,primid,groupid,Gd,Nd,primidd,groupidd,groupTot,OName,Size,arcsec_sep_from_bcg
0,LoVoCCS-5,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN
1,LoVoCCS-39,True,4.495,229.633396,6.401306,GALEX J151832.0+062404,6.382773e+18,6.382773e+18,AIS,230.070693,...,6.382773e+18,6382772985518885557,0.0,1.0,6.382773e+18,6382772985518885557,6382772985518885557,N,NaN,4.494224
2,LoVoCCS-55,True,NaN,NaN,NaN,None,NaN,NaN,None,NaN,...,NaN,None,NaN,NaN,NaN,None,None,None,NaN,NaN


Here we look at the number of matches found for this BCG candidate for each cluster:

In [40]:
pd.set_option('display.max_rows', None)
display(bcg4_cand_galex_nummatches.sort_values(ascending=False))
# Resetting the number of rows displayed
pd.set_option('display.max_rows', 15)

cluster_name
LoVoCCS-39    1
LoVoCCS-5     0
LoVoCCS-55    0
Name: count, dtype: object

Finally, list the clusters for which there is no match, and nominally there should be coverage:

In [41]:
bcg4_cand_galex_nomatch_clnames

array(['LoVoCCS-5', 'LoVoCCS-55'], dtype=object)